In [112]:
pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from google.colab import files
import io
import pandas as pd

In [ ]:
uploaded1 = files.upload()

Saving mock_train_set.csv to mock_train_set (1).csv


In [113]:
#Reading the given training csv file into a data frame
data = pd.read_csv(io.BytesIO(uploaded1['mock_train_set.csv']))
#Now converting data frame into a numpy array using df.values
train=data.values
#print (train)

X = np.array(train[:, :-1], requires_grad=False)
Y = np.array(train[:, -1], requires_grad=False)
Y = 2*Y - np.ones(len(Y))
#print (X)

In [114]:
dev = qml.device("default.qubit", wires=2)

In [115]:

def layer(W):
    qml.RX(W[0, 0], wires=0)
    qml.RX(W[0, 1], wires=0)
    qml.RX(W[0, 2], wires=0)
    qml.RX(W[1, 0], wires=1)
    qml.RX(W[1, 1], wires=1)
    qml.RX(W[1, 2], wires=1)
    qml.RZ(W[0, 0], wires=0)
    qml.RZ(W[0, 1], wires=0)
    qml.RZ(W[0, 2], wires=0)
    qml.RZ(W[1, 0], wires=1)
    qml.RZ(W[1, 1], wires=1)
    qml.RZ(W[1, 2], wires=1)
    qml.RY(W[0, 0], wires=0)
    qml.RY(W[0, 1], wires=0)
    qml.RY(W[0, 2], wires=0)
    qml.RY(W[1, 0], wires=1)
    qml.RY(W[1, 1], wires=1)
    qml.RY(W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])
def statepreparation(x):
    qml.AmplitudeEmbedding(features=x, wires=range(2), normalize =True)

In [116]:
@qml.qnode(dev)
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

In [117]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [118]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [119]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [120]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights,bias, x) for x in X]
    return square_loss(Y, predictions)

In [137]:
np.random.seed(0)
num_qubits = 2
num_layers = 7
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]]

 [[ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]]

 [[ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]

 [[ 0.02269755 -0.01454366  0.00045759]
  [-0.00187184  0.01532779  0.01469359]]

 [[ 0.00154947  0.00378163 -0.00887786]
  [-0.01980796 -0.00347912  0.00156349]]] 0.0


In [139]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 4

In [142]:
weights = weights_init
bias = bias_init
for it in range(10):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    #print (X_batch)
    Y_batch = Y[batch_index]
    #print (Y_batch)
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

Iter:     1 | Cost: 0.8171923 | Accuracy: 0.6666667 
Iter:     2 | Cost: 3.2291236 | Accuracy: 0.5233333 
Iter:     3 | Cost: 0.9597792 | Accuracy: 0.7466667 
Iter:     4 | Cost: 1.5642177 | Accuracy: 0.4766667 
Iter:     5 | Cost: 1.7429029 | Accuracy: 0.5766667 
Iter:     6 | Cost: 2.2471905 | Accuracy: 0.4366667 
Iter:     7 | Cost: 1.2662306 | Accuracy: 0.3866667 
Iter:     8 | Cost: 1.9547518 | Accuracy: 0.1733333 
Iter:     9 | Cost: 1.5926295 | Accuracy: 0.1966667 
Iter:    10 | Cost: 1.3634367 | Accuracy: 0.4766667 


In [129]:
upload2=files.upload()

Saving mock_test_set.csv to mock_test_set (1).csv


In [131]:
data_1= pd.read_csv(io.BytesIO(upload2['mock_test_set.csv']))
test=data_1.values
#print (train)

X_test = np.array(test[:, :-1], requires_grad=False)
Y_test = np.array(test[:, -1], requires_grad=False)
Y_test = 2*Y_test - np.ones(len(Y_test))
#print (X_test)

In [143]:

# Compute accuracy
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
acc = accuracy(Y_test, predictions)

print(
    " Cost: {:0.7f} | Accuracy: {:0.7f} ".format( cost(weights, bias, X, Y), acc)
)

 Cost: 1.3634367 | Accuracy: 0.5166667 
